In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [7]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230126_Plate527/"

if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)

all_folders_drop, excel_drop = get_dropbox_video_folders("/DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate527/", True)
# folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "452"]
# all_folders_drop = pd.read_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# for xl_adress in excel_drop:
#     file_name = xl_adress.split('/')[-1]
    
#     download(xl_adress, directory_targ+file_name)
clear_output(wait=False)

all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230126_Plate527_01,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,01
1,20230126_Plate527_02,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,02
2,20230126_Plate527_03,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,03
3,20230126_Plate527_04,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,04
4,20230126_Plate527_05,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,05
5,20230126_Plate527_06,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,06
6,20230126_Plate527_07,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,07
7,20230126_Plate527_08,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,08
8,20230126_Plate527_09,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,09
9,20230126_Plate527_10,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,10


In [8]:
# directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_dataset/"
# all_folders_drop.to_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# save_dropbox_state("/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate552/", is_video=True)

folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"].ge("527")]
# folders_drop = folders_drop.loc[folders_drop["Date Imaged"] == "20221208"]

In [9]:
folders_drop = folders_drop.sort_values(by=['folder'])


In [10]:
# FINAL CHECK

print(directory_targ)
folders_drop

/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230126_Plate527/


,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230126_Plate527_01,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,01
1,20230126_Plate527_02,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,02
2,20230126_Plate527_03,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,03
3,20230126_Plate527_04,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,04
4,20230126_Plate527_05,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,05
5,20230126_Plate527_06,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,06
6,20230126_Plate527_07,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,07
7,20230126_Plate527_08,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,08
8,20230126_Plate527_09,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,09
9,20230126_Plate527_10,527,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,10


In [11]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    16,
    "10:00:00",
    "transfer_test"
)


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2781047


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2781048
Submitted batch job 2781049


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


In [3]:
all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230126_Plate528_01,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,01
1,20230126_Plate528_02,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,02
2,20230126_Plate528_03,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,03
3,20230126_Plate528_04,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,04
4,20230126_Plate528_05,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,05
5,20230126_Plate528_06,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,06
6,20230126_Plate528_07,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,07
7,20230126_Plate528_08,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,08
8,20230126_Plate528_09,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,09
9,20230126_Plate528_10,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,10


In [4]:
folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "528"]
folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230126_Plate528_01,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,01
1,20230126_Plate528_02,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,02
2,20230126_Plate528_03,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,03
3,20230126_Plate528_04,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,04
4,20230126_Plate528_05,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,05
5,20230126_Plate528_06,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,06
6,20230126_Plate528_07,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,07
7,20230126_Plate528_08,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,08
8,20230126_Plate528_09,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,09
9,20230126_Plate528_10,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,10


In [5]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230201_Plate528/"

In [7]:
for xl_adress in excel_drop:
    file_name = xl_adress.split('/')[-1]
    
    download(xl_adress, directory_targ+file_name)

2023-04-11 11:22:49,196-[INFO]- dropbox:390 -> Refreshing access token.
2023-04-11 11:22:49,395-[INFO]- dropbox:474 -> Request to files/download


In [8]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    16,
    "10:00:00",
    "transfer_test"
)
clear_output(wait=False)